In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)


In [2]:
num_drones = 200

drone_ids = [f"HC-DRN-{i:04d}" for i in range(1, num_drones + 1)]

drone_models = ["HC-Scout", "HC-Sentinel", "HC-Ranger"]


In [3]:
manufacturing_df = pd.DataFrame({
    "Drone_ID": drone_ids,
    "Drone_Model": np.random.choice(drone_models, num_drones, p=[0.4, 0.35, 0.25]),
    "Component_Type": np.random.choice(
        ["Motor", "Battery", "GPS", "Camera", "Flight Controller"], num_drones
    ),
    "Supplier": np.random.choice(
        ["Supplier-A", "Supplier-B", "Supplier-C", "Supplier-D"], num_drones
    ),
    "Batch_No": np.random.choice(
        ["BATCH-01", "BATCH-02", "BATCH-03"], num_drones
    ),
    "Manufacturing_Date": [
        datetime.today() - timedelta(days=int(x))
        for x in np.random.randint(1, 365, num_drones)
    ]
})


In [4]:
defect_flag = np.random.choice([0, 1], num_drones, p=[0.78, 0.22])

defect_type = []
rework_cost = []

for comp, flag in zip(manufacturing_df["Component_Type"], defect_flag):
    if flag == 0:
        defect_type.append("None")
        rework_cost.append(0)
    else:
        if comp == "Battery":
            defect_type.append("Overheating")
        elif comp == "GPS":
            defect_type.append("Signal Loss")
        elif comp == "Motor":
            defect_type.append("Calibration Issue")
        else:
            defect_type.append("Assembly Fault")
        rework_cost.append(np.random.randint(3000, 15000))

manufacturing_df["Defect_Flag"] = defect_flag
manufacturing_df["Defect_Type"] = defect_type
manufacturing_df["Rework_Cost_INR"] = rework_cost


In [5]:
manufacturing_df.to_csv("01_drone_manufacturing.csv", index=False)

manufacturing_df.head()


,Drone_ID,Drone_Model,Component_Type,Supplier,Batch_No,Manufacturing_Date,Defect_Flag,Defect_Type,Rework_Cost_INR
0,HC-DRN-0001,HC-Scout,Camera,Supplier-A,BATCH-01,2025-09-14 15:22:28.485776,0,None,0
1,HC-DRN-0002,HC-Ranger,GPS,Supplier-D,BATCH-01,2025-07-09 15:22:28.485776,0,None,0
2,HC-DRN-0003,HC-Sentinel,Motor,Supplier-B,BATCH-01,2025-10-16 15:22:28.485776,0,None,0
3,HC-DRN-0004,HC-Sentinel,Camera,Supplier-C,BATCH-02,2025-08-16 15:22:28.485776,1,Assembly Fault,11428
4,HC-DRN-0005,HC-Scout,Camera,Supplier-D,BATCH-01,2025-10-26 15:22:28.485776,0,None,0


In [6]:
flight_df = pd.DataFrame({
    "Drone_ID": manufacturing_df["Drone_ID"],
    "Total_Flight_Hours": np.round(np.random.uniform(20, 600, num_drones), 1),
    "Avg_Altitude_m": np.random.randint(60, 350, num_drones),
    "Battery_Health_%": np.round(np.random.uniform(60, 100, num_drones), 1),
    "Motor_Temperature_C": np.random.randint(45, 95, num_drones),
    "Signal_Strength_%": np.random.randint(65, 100, num_drones)
})


In [7]:
failure_risk = []

for _, row in flight_df.iterrows():
    if row["Battery_Health_%"] < 70 or row["Motor_Temperature_C"] > 85:
        failure_risk.append("High")
    elif row["Signal_Strength_%"] < 75:
        failure_risk.append("Medium")
    else:
        failure_risk.append("Low")

flight_df["Failure_Risk"] = failure_risk


In [8]:
flight_df = flight_df.merge(
    manufacturing_df[["Drone_ID", "Drone_Model"]],
    on="Drone_ID",
    how="left"
)


In [9]:
flight_df.to_csv("02_drone_flight_performance.csv", index=False)

flight_df.head()


,Drone_ID,Total_Flight_Hours,Avg_Altitude_m,Battery_Health_%,Motor_Temperature_C,Signal_Strength_%,Failure_Risk,Drone_Model
0,HC-DRN-0001,264.8,192,87.2,75,68,Medium,HC-Scout
1,HC-DRN-0002,225.2,294,62.9,88,82,High,HC-Ranger
2,HC-DRN-0003,576.0,104,61.2,54,70,High,HC-Sentinel
3,HC-DRN-0004,126.9,308,70.3,83,99,Low,HC-Sentinel
4,HC-DRN-0005,72.4,90,78.5,57,86,Low,HC-Scout


In [10]:
supply_chain_df = pd.DataFrame({
    "Drone_ID": manufacturing_df["Drone_ID"],
    "Component": np.random.choice(
        ["Battery Pack", "Motor Unit", "Camera Module", "GPS Module"],
        num_drones
    ),
    "Supplier": np.random.choice(
        ["Supplier-A", "Supplier-B", "Supplier-C", "Supplier-D"],
        num_drones
    ),
    "Lead_Time_Days": np.random.randint(5, 50, num_drones),
    "Inventory_Level": np.random.randint(5, 250, num_drones),
    "Procurement_Cost_INR": np.random.randint(6000, 55000, num_drones)
})


In [11]:
stockout_risk = []

for _, row in supply_chain_df.iterrows():
    if row["Inventory_Level"] < 25 and row["Lead_Time_Days"] > 35:
        stockout_risk.append("High")
    elif row["Inventory_Level"] < 60:
        stockout_risk.append("Medium")
    else:
        stockout_risk.append("Low")

supply_chain_df["Stockout_Risk"] = stockout_risk


In [12]:
supply_chain_df = supply_chain_df.merge(
    manufacturing_df[["Drone_ID", "Drone_Model"]],
    on="Drone_ID",
    how="left"
)


In [13]:
supply_chain_df.to_csv("03_drone_supply_chain.csv", index=False)

supply_chain_df.head()


,Drone_ID,Component,Supplier,Lead_Time_Days,Inventory_Level,Procurement_Cost_INR,Stockout_Risk,Drone_Model
0,HC-DRN-0001,GPS Module,Supplier-D,25,190,52546,Low,HC-Scout
1,HC-DRN-0002,GPS Module,Supplier-B,16,157,25835,Low,HC-Ranger
2,HC-DRN-0003,GPS Module,Supplier-B,45,51,35033,Medium,HC-Sentinel
3,HC-DRN-0004,GPS Module,Supplier-B,37,42,30000,Medium,HC-Sentinel
4,HC-DRN-0005,Motor Unit,Supplier-C,8,147,51496,Low,HC-Scout


In [14]:
!pip install psycopg2-binary sqlalchemy


Defaulting to user installation because normal site-packages is not writeable


In [18]:
from sqlalchemy import create_engine

username = "postgres"
password = "MyPass%402025"
host = "localhost"
port = "5432"
database = "hc_robotics"

engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

In [19]:
manufacturing_df.to_sql(
    "manufacturing", engine, if_exists="replace", index=False
)

flight_df.to_sql(
    "flight_performance", engine, if_exists="replace", index=False
)

supply_chain_df.to_sql(
    "supply_chain", engine, if_exists="replace", index=False
)


200